# <b> Glacier runoff model for La Paz, Bolivia (revised 1/5/23)

Molly Arndt, Middlebury College

Last update: 1/29/23

## <b> Step 1: Import necessary software packages for graphing and visualization

In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import shapely.geometry as shpg

## <b> Step 2: import and set up OGGM software

In [2]:
from oggm import cfg, utils, workflow, tasks, graphics

In [3]:
# place where output is saved
path = '/home/ulteelab/Documents/Molly-thesis/OGGM_output'

In [4]:
# work with parameters 
cfg.initialize(logging_level='WARNING')
cfg.PATHS['working_dir']=path
#cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGMHydro')
#Give this a full file path (see installation and setup) (on agnesi) /home/ulteelab/OGGM

cfg.PARAMS['store_model_geometry'] = True
cfg.PARAMS['use_multiprocessing'] = True
cfg.PARAMS['continue_on_error']=True #this is to avoid issues with the files with invalide geometries

#You can provide any other dataset to OGGM by setting the climate_file parameter in params.cfg

2023-03-09 13:13:59: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-03-09 13:13:59: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-03-09 13:13:59: oggm.cfg: Multiprocessing: using all available processors (N=12)
2023-03-09 13:14:00: oggm.cfg: PARAMS['store_model_geometry'] changed from `False` to `True`.
2023-03-09 13:14:00: oggm.cfg: Multiprocessing switched ON after user settings.
2023-03-09 13:14:00: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.


## Step 3: Define glaciers for the run

In [5]:
# Download glacier outlines
utils.get_rgi_dir(version='62')  # path to the data after download

'/home/ulteelab/OGGM/rgi/RGIV62'

In [6]:
# Get rgi region file for region 16 (South America)
fr = utils.get_rgi_region_file(16, version='62')

In [7]:
# Read file as as geopandas
gdf = gpd.read_file(fr)

In [8]:
# Add basin shapefile
path = '~/Documents/Molly-thesis/Data/Basin_shapefiles/output/LaPaz_dissolved.shp' 
basin = gpd.read_file(path)

In [9]:
# Select glaciers within the basin
in_bas = [basin.geometry.contains(shpg.Point(x, y))[0] for
          (x, y) in zip(gdf.CenLon, gdf.CenLat)]
gdf_sel = gdf.loc[in_bas]

In [10]:
# Select rgi id's from the gdf_sel file
rgi_ids = gdf_sel['RGIId']

## Step 4. Prepare glacier data

In [14]:
# Initiate glacier directory task for each glacier
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/L3-L5_files/CRU/centerlines/qc3/pcp2.5/no_match'
gdir = workflow.init_glacier_directories(rgi_ids, from_prepro_level=5, prepro_border=80, prepro_base_url=base_url) 

2023-03-09 13:16:24: oggm.workflow: init_glacier_directories from prepro level 5 on 35 glaciers.
2023-03-09 13:16:24: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 35 glaciers


## Step 5. Initiate climate runs

### I. Run constant climate commitment run

In [ ]:
# Initiate run_with_hydro using constant climate
for g in gdir:
    file_id = '_ct'
    tasks.run_with_hydro(g, run_task=tasks.run_constant_climate, nyears=100, y0=2014, halfsize=5, store_monthly_hydro=True, 
                     output_filesuffix=file_id);

In [ ]:
# Export data from constant climate run and compile run output
import datetime
output_path = '~/Documents/Molly-thesis/Data/{}-ct_compiled_output.nc'.format(datetime.date.today()) 
new_ds = utils.compile_run_output(gdir, input_filesuffix=file_id, path=output_path)

In [ ]:
# Load in the glacier data from above (only when data has been saved)
export_date = 'xxx'
input_path = '~/Documents/Molly-thesis/Data/{}-ct_compiled_output.nc'.format(export_date)
ct_ds = xr.open_dataset(input_path)

In [ ]:
# Look at the new dataset
ct_ds

### II. Run CMIP5 projection run

### a. Historical run for reference climate period

In [12]:
# Run the historical period with a fixed geometry spinup
for g in gdir:
    file_id = '_hist_hydro'
    tasks.run_with_hydro(g, run_task=tasks.run_from_climate_data,
                         fixed_geometry_spinup_yr=1990,  # this tells to do spinup
                         ref_area_from_y0=True,  # 
                         store_monthly_hydro=True,
                         output_filesuffix=file_id);
# I think i have to run this every time I start the kernel again so file_id hist_hydro exists for gcm use

2023-01-29 20:42:01: oggm.core.flowline: InvalidWorkflowError occurred during task run_from_climate_data_hist_hydro on RGI60-16.00505: Need a valid `model_flowlines` file. If you explicitly want to use `inversion_flowlines`, set use_inversion_flowlines=True.
2023-01-29 20:42:01: oggm.core.flowline: InvalidWorkflowError occurred during task run_with_hydro_hist_hydro on RGI60-16.00505: The run task (run_from_climate_data) did not run successfully.
2023-01-29 20:42:01: oggm.core.flowline: InvalidWorkflowError occurred during task run_from_climate_data_hist_hydro on RGI60-16.00506: Need a valid `model_flowlines` file. If you explicitly want to use `inversion_flowlines`, set use_inversion_flowlines=True.
2023-01-29 20:42:01: oggm.core.flowline: InvalidWorkflowError occurred during task run_with_hydro_hist_hydro on RGI60-16.00506: The run task (run_from_climate_data) did not run successfully.
2023-01-29 20:42:03: oggm.core.flowline: InvalidWorkflowError occurred during task run_from_climate_

In [ ]:
# Export data from historic climate run and compile run output
import datetime
output_path = '~/Documents/Molly-thesis/Data/{}-hist_hydro_compiled_output.nc'.format(datetime.date.today()) 
new_ds = utils.compile_run_output(gdir, input_filesuffix=file_id, path=output_path)

In [ ]:
# Load in the glacier data from above (only when data has been saved)
export_date = 'xxxx'
input_path_hist = '~/Documents/Molly-thesis/Data/{}}-hist_hydro_compiled_output.nc'.format(export_date)
hist_ds = xr.open_dataset(input_path_hist)

### b. Projection run

<b> Remember: you need to change the GCM for bp and bt

In [33]:
# Process climate data for each GCM 

# name the gcm that is currently used
current_gcm = 'NorESM1-M'

# identify which gcms ('poorly behaved') have only 3 rcp scenarios instead of 4
poorly_behaved = ['CNRM-CM5', 'CanESM2', 'MPI-ESM-LR']

# import climate data from where it is stored within oggm
## Remember to change name of gcm each time this is run! 
## Note: I have tried to make this more efficient, but can't get the {} to work right
from oggm.shop import gcm_climate
bp = 'https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/pr/pr_mon_{}_{}_r1i1p1_g025.nc' #precipitation
bt = 'https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/tas/tas_mon_{}_{}_r1i1p1_g025.nc' #temperature

# process climate data for each rcp scenario
if current_gcm in poorly_behaved:
    for rcp in ['rcp26', 'rcp45', 'rcp85']:
        # Download the files
        ft = utils.file_downloader(bt.format(current_gcm, rcp))
        fp = utils.file_downloader(bp.format(current_gcm, rcp))
        # bias correct them
        for g in gdir:
            workflow.execute_entity_task(gcm_climate.process_cmip_data, [g],
                                         filesuffix='_{}_{}'.format(current_gcm, rcp),  # recognize the climate file for later
                                         fpath_temp=ft,  # temperature projections
                                         fpath_precip=fp,  # precip projections
                                             );
else:
    for rcp in ['rcp26', 'rcp45', 'rcp60', 'rcp85']:
        # Download the files
        ft = utils.file_downloader(bt.format(current_gcm, rcp))
        fp = utils.file_downloader(bp.format(current_gcm, rcp))
        # bias correct them
        for g in gdir:
            workflow.execute_entity_task(gcm_climate.process_cmip_data, [g],
                                         filesuffix='_{}_{}'.format(current_gcm, rcp),  # recognize the climate file for later
                                         fpath_temp=ft,  # temperature projections
                                         fpath_precip=fp,  # precip projections
                                             );

2023-01-29 21:22:52: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2023-01-29 21:22:52: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2023-01-29 21:22:53: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2023-01-29 21:22:54: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2023-01-29 21:22:55: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2023-01-29 21:22:56: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2023-01-29 21:22:57: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2023-01-29 21:22:57: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2023-01-29 21:22:58: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2023-01-29 21:22:59: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers
2023-01-29 21:23:00: oggm.workflow: Execute entity tasks [process_cmip_data] on 1 glaciers

In [34]:
# Run the model with hydrologic output for each rcp scenario
if current_gcm in poorly_behaved:
    for g in gdir:
        for rcp in ['rcp26', 'rcp45', 'rcp85']:
            rid = '_{}_{}'.format(current_gcm, rcp)
            tasks.run_with_hydro(g, run_task=tasks.run_from_climate_data,
                                 climate_filename='gcm_data',  # use gcm_data, not climate_historical
                                 climate_input_filesuffix=rid,  # use the chosen scenario (gcm and rcp)
                                 init_model_filesuffix='_hist_hydro',  # start using hist data
                                 ref_geometry_filesuffix='_hist_hydro',  # also use this as area reference
                                 ref_area_from_y0=True,  # and keep the same reference area as for the historical simulations
                                 output_filesuffix=rid,  # recognize the run for later
                                 store_monthly_hydro=True,  # add monthly diagnostics
                                 );
else:
    for g in gdir:
        for rcp in ['rcp26', 'rcp45', 'rcp60', 'rcp85']:
            rid = '_{}_{}'.format(current_gcm, rcp)
            tasks.run_with_hydro(g, run_task=tasks.run_from_climate_data,
                                 climate_filename='gcm_data',
                                 climate_input_filesuffix=rid, 
                                 init_model_filesuffix='_hist_hydro', 
                                 ref_geometry_filesuffix='_hist_hydro',  
                                 ref_area_from_y0=True, 
                                 output_filesuffix=rid, 
                                 store_monthly_hydro=True, 
                                 );

2023-01-29 21:25:20: oggm.core.flowline: FileNotFoundError occurred during task run_from_climate_data_NorESM1-M_rcp26 on RGI60-16.00505: [Errno 2] No such file or directory: b'/home/ulteelab/Documents/Molly-thesis/OGGM_output/per_glacier/RGI60-16/RGI60-16.00/RGI60-16.00505/model_geometry_hist_hydro.nc'
2023-01-29 21:25:20: oggm.core.flowline: InvalidWorkflowError occurred during task run_with_hydro_NorESM1-M_rcp26 on RGI60-16.00505: The run task (run_from_climate_data) did not run successfully.
2023-01-29 21:25:20: oggm.core.flowline: FileNotFoundError occurred during task run_from_climate_data_NorESM1-M_rcp45 on RGI60-16.00505: [Errno 2] No such file or directory: b'/home/ulteelab/Documents/Molly-thesis/OGGM_output/per_glacier/RGI60-16/RGI60-16.00/RGI60-16.00505/model_geometry_hist_hydro.nc'
2023-01-29 21:25:20: oggm.core.flowline: InvalidWorkflowError occurred during task run_with_hydro_NorESM1-M_rcp45 on RGI60-16.00505: The run task (run_from_climate_data) did not run successfully.


In [35]:
new_ds_nor = utils.compile_run_output(gdir, input_filesuffix=rid)
new_ds_nor

2023-01-29 21:27:03: oggm.utils: Applying global task compile_run_output on 35 glaciers
2023-01-29 21:27:03: oggm.utils: Applying compile_run_output on 35 gdirs.


<xarray.Dataset>
Dimensions:                       (time: 82, rgi_id: 35, month_2d: 12)
Coordinates:
  * time                          (time) float64 2.02e+03 ... 2.101e+03
  * rgi_id                        (rgi_id) <U14 'RGI60-16.00495' ... 'RGI60-1...
    hydro_year                    (time) int64 2020 2021 2022 ... 2099 2100 2101
    hydro_month                   (time) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    calendar_year                 (time) int64 2019 2020 2021 ... 2098 2099 2100
    calendar_month                (time) int64 4 4 4 4 4 4 4 4 ... 4 4 4 4 4 4 4
  * month_2d                      (month_2d) int64 1 2 3 4 5 6 7 8 9 10 11 12
    calendar_month_2d             (month_2d) int64 4 5 6 7 8 9 10 11 12 1 2 3
Data variables: (12/24)
    volume                        (time, rgi_id) float64 0.0 2.76e+04 ... 0.0
    volume_bsl                    (time, rgi_id) float64 0.0 0.0 0.0 ... 0.0 0.0
    volume_bwl                    (time, rgi_id) float64 0.0 0.0 0.0 ... 0.0 0.0
    area                          (time, rgi_id) float64 0.0 1.219e+04 ... 0.0
    length                        (time, rgi_id) float64 0.0 30.0 ... 0.0 0.0
    calving                       (time, rgi_id) float64 0.0 0.0 0.0 ... 0.0 0.0
    ...                            ...
    liq_prcp_on_glacier_monthly   (time, month_2d, rgi_id) float64 0.0 ... nan
    snowfall_off_glacier_monthly  (time, month_2d, rgi_id) float64 3.545e+04 ...
    snowfall_on_glacier_monthly   (time, month_2d, rgi_id) float64 0.0 ... nan
    water_level                   (rgi_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    glen_a                        (rgi_id) float64 1.297e-24 ... 1.297e-24
    fs                            (rgi_id) float64 5.7e-20 5.7e-20 ... 5.7e-20
Attributes:
    description:    OGGM model output
    oggm_version:   1.5.3
    calendar:       365-day no leap
    creation_date:  2023-01-30 02:27:03

In [ ]:
# Export data from climate run and compile run output
import datetime
if current_gcm in poorly_behaved:
    for rcp in ['rcp26', 'rcp45', 'rcp85']:
        rid = '_{}_{}'.format(current_gcm, rcp) #same rid as in step above
        output_path = '~/Documents/Molly-thesis/Data/{}-gcm_data_{}_{}_compiled_output.nc'.format(datetime.date.today(), current_gcm, rcp) #define name for file 
        new_ds = utils.compile_run_output(gdir, input_filesuffix=rid, path=output_path) #compile all rcp outputs into one file
else:
    for rcp in ['rcp26', 'rcp45', 'rcp60', 'rcp85']:
        rid = '_{}_{}'.format(current_gcm, rcp)
        output_path = '~/Documents/Molly-thesis/Data/{}-gcm_data_{}_{}_compiled_output.nc'.format(datetime.date.today(), current_gcm, rcp) 
        new_ds = utils.compile_run_output(gdir, input_filesuffix=rid, path=output_path)

# Step 6: Look at seasonality

Note: this is still experimental

In [ ]:
# Select only the runoff variables and convert them to megatonnes (instead of kg)
monthly_runoff = new_ds['melt_off_glacier_monthly'].sum(dim='rgi_id') + new_ds['melt_on_glacier_monthly'].sum(dim='rgi_id') + new_ds['liq_prcp_off_glacier_monthly'].sum(dim='rgi_id') + new_ds['liq_prcp_on_glacier_monthly'].sum(dim='rgi_id') 
monthly_runoff *= 1e-9
monthly_runoff.clip(0).plot(cmap='Blues', cbar_kwargs={'label':'Mt'}); plt.xlabel('Months'); plt.ylabel('Years');

In [ ]:
# This should work in both hemispheres maybe?
#ds called new_ds
ds_roll = new_ds.roll(month_2d=new_ds['calendar_month_2d'].data[0]-1, roll_coords=True)
ds_roll['month_2d'] = ds_roll['calendar_month_2d']

# Select only the runoff variables and convert them to megatonnes (instead of kg)
monthly_runoff = ds_roll['melt_off_glacier_monthly'].sum(dim='rgi_id') + ds_roll['melt_on_glacier_monthly'].sum(dim='rgi_id')+ ds_roll['liq_prcp_off_glacier_monthly'].sum(dim='rgi_id') + ds_roll['liq_prcp_on_glacier_monthly'].sum(dim='rgi_id')
monthly_runoff *= 1e-9
monthly_runoff.clip(0).plot(cmap='Blues', cbar_kwargs={'label':'Mt'}); plt.xlabel('Months'); plt.ylabel('Years');

In [ ]:
monthly_runoff.sel(time=[0, 30, 99]).plot(hue='time'); plt.title('Annual cycle'); plt.xlabel('Month'); plt.ylabel('Runoff (Mt)');

In [ ]:
# Pick the variables we need (the 2d ones)
sel_vars = [v for v in ds_roll.variables if 'month_2d' in ds_roll[v].dims]

# Pick the first decade and average it
df_m_s = ds_roll[sel_vars].isel(time=slice(0, 10)).mean(dim='time').to_dataframe() * 1e-9
# Rename the columns for readability
df_m_s.columns = [c.replace('_monthly', '') for c in df_m_s.columns]
# Because of floating point precision sometimes runoff can be slightly below zero, clip
df_m_s = df_m_s.clip(0)

# Same for end
df_m_e = ds_roll[sel_vars].isel(time=slice(-11, -1)).mean(dim='time').to_dataframe() * 1e-9
df_m_e.columns = [c.replace('_monthly', '') for c in df_m_s.columns]
df_m_e = df_m_e.clip(0)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True);
df_m_s[regional_sum_nonzero].plot.area(ax=ax1, legend=False, title='Year 0-10', color=sns.color_palette("rocket"));
df_m_e[runoff_vars].plot.area(ax=ax2, title='Year 90-100', color=sns.color_palette("rocket"));
ax1.set_ylabel('Monthly runoff (Mt)'); ax1.set_xlabel('Month'); ax2.set_xlabel('Month');